In [1]:
import glob
import pandas as pd
import re
import nilearn as nil
from nilearn import *
from nilearn import image
from os.path import basename
import numpy as np
import yaml
from yaml.loader import SafeLoader
from socket import gethostname

from get_all_series import get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed
from get_all_series import get_all_subj_df

In [2]:
gethostname()

'Benjamins-MacBook-Pro-2.local'

In [3]:
# Open the file and load the file
with open('config.yml') as f:
    all_yaml = yaml.load(f, Loader=SafeLoader)
    if gethostname() in all_yaml.keys():
        config = all_yaml[gethostname()]
    else:
        config = all_yaml['default']

In [4]:
config

{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/',
 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/',
 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics',
 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/',
 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions',
 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/',
 'fmriprep_dir': '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/',
 'nii_raw_path': 'sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii',
 'mask_location': '/Users/benjaminsmith/Google Drive

get all series

get masks

iterate through series

iterate through masks

produce the vector for this mask/series

for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;
this may be slightly different for Go and Stop trials. For Stop trials it is simply the moment of the tone;
for Go trials it's harder to specify.

for each TRIAL, you will end up with a data table with columns (a) time from trial class reveal to image; (b) separate columns for measurement of each ROI
these can be concatenated across trials, runs, and subjects

then create n_{ROI}*n_{trial classes} graphs of samples, where x is the time from trial class reveal to image, and y is the measurement. plot a lowess curve.

potentially overlay the lowess curves for each trial type so that the response within each ROI can be easily compared.


In [5]:
# #need to replace these with references to the config yml
# dropbox_data_dir = '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/'
# fmriprep_dir = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/'
# nii_raw_path = 'sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii'
# mask_location = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/striatum/'
dropbox_data_dir = config['dropbox_data_dir']
fmriprep_dir = config['fmriprep_dir']
nii_raw_path = config['nii_raw_path']
mask_location = config['mask_location'] + 'striatum'

get all series

In [6]:
glob_path = fmriprep_dir + nii_raw_path
print("looking up " + glob_path)
nii_raw_files = glob.glob(glob_path)

looking up /Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii


In [7]:
print(nii_raw_files)

['/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV285/ses-wave1/func/s6_sub-DEV285_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV004/ses-wave1/func/s6_sub-DEV004_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV293/ses-wave1/func/s6_sub-DEV293_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii']


get masks

In [8]:
mask_paths = glob.glob(mask_location+"*.nii.gz")
mask_labels = [re.match(".*/(.*)\.nii\.gz",mp)[1] for mp in mask_paths]

mask_df = pd.DataFrame({
    'mask_label':mask_labels,
    'mask_path': mask_paths
})

iterate through series

In [11]:
#mark the number of seconds each scan takes.
#TR = 2.0
roi_data = get_roi_data(nii_raw_files,mask_df)



DEV285, 1
DEV004, 1
DEV293, 1


In [12]:
roi_data is None

False

In [23]:
for s in roi_data.keys():
    print(s)
    for wave in roi_data[s].keys():
        print(wave)
        print(roi_data[s][wave].shape)
        print(roi_data[s][wave].columns)

DEV285
1
(212, 0)
Index([], dtype='object')
DEV004
1
(207, 0)
Index([], dtype='object')
DEV293
1
(212, 0)
Index([], dtype='object')


In [30]:
sst_all_behavioral_data = pd.read_csv(dropbox_data_dir + "sst_behavioral_data_all.csv")



for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;

In [31]:


sst_all_behavioral_data = get_behavioral_data_with_moment_trial_type_revealed(sst_all_behavioral_data)



In [32]:
sst_all_behavioral_data['class_type_reveal_onset'] = sst_all_behavioral_data.class_type_reveal + sst_all_behavioral_data.onset
sst_all_behavioral_data.loc[sst_all_behavioral_data.go_no_go_condition_label=='Cue','class_type_reveal_onset']=None

Now we need a list of the TRs by their time of appearance; that way we can mark which ones are relevant to the ROIs for each trial.

For now, let's assume that the timing of the TRs matches the timing in the matlab files, but this MUST be checked.

In [ ]:
# run_length = 207
# rt_time_list = [x*2.0 for x in range(run_length)]

these are going to be generated dynamically based on the run length

In [35]:
#from get_all_series import get_all_subj_df
all_subj_df = get_all_subj_df(roi_data, sst_all_behavioral_data)

DEV285
DEV285, 1:212
DEV004
DEV004, 1:207
DEV293
DEV293, 1:212


In [36]:
all_subj_df.groupby(['trial_n','subid','wave']).offset.count()

trial_n  subid   wave
1        DEV004  1       12
         DEV285  1       12
         DEV293  1       12
3        DEV004  1       12
         DEV285  1       13
                         ..
253      DEV285  1        9
         DEV293  1        9
255      DEV004  1        1
         DEV285  1        7
         DEV293  1        7
Name: offset, Length: 384, dtype: int64

In [37]:
all_subj_df.to_csv(dropbox_data_dir + 'SST_roi_by_time_point.csv')

We should have up to 420 or so.

In [38]:
sst_all_behavioral_data.loc[sst_all_behavioral_data.subid=='DEV293'][['trial_n','onset','class_type_reveal_onset','condition']]

,trial_n,onset,class_type_reveal_onset,condition
248832,1,0.011194,2.011194,FailedStop
248833,2,0.000000,NaN,Cue
248834,3,2.763036,4.763036,CorrectGo
248835,4,0.000000,NaN,Cue
248836,5,5.528667,7.528667,CorrectGo
...,...,...,...,...
249083,252,0.000000,NaN,Cue
249084,253,414.584341,414.885800,CorrectGo
249085,254,0.000000,NaN,Cue
249086,255,417.849799,418.151915,CorrectStop


In [ ]:

# for each TRIAL, you will end up with a data table with columns (a) time from trial class reveal to image; (b) separate columns for measurement of each ROI
# these can be concatenated across trials, runs, and subjects

# then create n_{ROI}*n_{trial classes} graphs of samples, where x is the time from trial class reveal to image, and y is the measurement. plot a lowess curve.

# potentially overlay the lowess curves for each trial type so that the response within each ROI can be easily compared.
